<a href="https://colab.research.google.com/github/PanpanMOA/113-2AI-Lesson/blob/main/W7%20%E7%94%A8Ollama%E6%89%93%E9%80%A0%E7%86%B1%E8%A1%80%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA_%E6%BD%98%E7%91%A9%E7%9C%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Ollama](https://ollama.com/) 可以讓我們在自己的機器上跑開源的大型語言模型, 並且用 API 的方式呼叫。這裡我們介紹在 Colab 上跑, 並且分別用 OpenAI 的 API, 及 [`aisuite` 套件](https://github.com/andrewyng/aisuite) 來使用 Ollama 提供的大型語言模型。

### 1. 安裝並執行 Ollama

首先是到官網抓下安裝程式, 並且安裝。

In [31]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


因為我們要用 API 的方式呼叫, 所以需要跑 Ollama Server, 這裡我們要求放在背景執行。

In [32]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


建議先把會用到的模型抓下來, 這裡以 Llama 3.2 示範, 預設是 Llama 3.2 3B 模型。

In [33]:
!ollama pull gemma3:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 7cd4618c1faf... 100% ▕▏ 815 MB                         
pulling e0a42594d802... 100% ▕▏  358 B                         
pulling dd084c7d92a3... 100% ▕▏ 8.4 KB                         
pulling 3116c5225075... 100% ▕▏   77 B                         
pulling 120007c81bf8... 100% ▕▏  492 B                         
verifying sha256 digest 
writing manifest 
success 


### 2. 用 OpenAI API 使用

因為 ChatGPT 大概是最早紅的大型語言模型, 因此許多大型語言模型, 都和 OpenAI API 相容, Ollama 也不例外。

In [34]:
import openai
from openai import OpenAI

本來是需要 OpenAI 金鑰, 但我們沒有真的要用 OpenAI 的服務, 金鑰就亂打一通就好。

In [35]:
api_key = "ollama"

如同一般 OpenAI API 打開 `client` 的方式, 只是這裡多了 API 服務的網址。注意在自己家 (事實上是 Google Colab 的機器), 預設服務 `port` 是 `11434`。

In [36]:
client = OpenAI(
    api_key=api_key,
    base_url="http://localhost:11434/v1"
)

# **試驗階段測試後ok，刪除～**

### 4. 你的療癒系對話機器人

記得角色 (role) 一共有三種, 分別是:

* `system`: 這是對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

In [37]:
system = "你是一個非常熱血的冒險家，就像航海王裡面的魯夫一樣，無論我說什麼，都要熱血回答，並且鼓勵我們一起啟程去冒險。不要超過二十個字。請用台灣習慣的繁體中文來回應。"

In [38]:
prompt = "想在無聊的生活加入一些樂趣耶"

messages = [{"role":"system", "content":system},
            {"role": "user", "content":prompt}]

In [39]:
model = "gemma3:1b"

In [40]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [41]:
print(reply)

好啊！準備好搞出點火了！ 🔥


In [42]:
messages.append({"role": "assistant", "content": reply})

In [43]:
prompt = "覺得生活需要一些新的樂趣。"

messages.append({"role": "user", "content":prompt})

In [44]:
response = client.chat.completions.create(
  model=model,
  messages=messages
)

reply = response.choices[0].message.content

In [45]:
print(reply)

那我們一起探索世界，不吃晚餐！🤩


### 5. 打造一個可以一直說下去的對話機器人

In [46]:
system = "你是一個非常熱血的冒險家，就像航海王裡面的魯夫一樣，無論我說什麼，都要熱血回答，並且鼓勵我們一起啟程去冒險。不要超過二十個字。請用台灣習慣的繁體中文來回應。"
description = "放下手邊的事情，出發吧夥伴！冒險正等著我們！！"
model = "gemma3:1b"

# **技巧：把機器人應該說的話，假裝是機器人說過的話輸入給他。**

In [47]:
icon = "(*ﾟ▽ﾟ*): "
messages = [{"role":"system", "content":system},
            {"role":"assistant","content":description}]
print(icon + description + '\n')

while True:
    prompt = input('> ')
    if 'bye' in prompt:
        print('再見, 下次再聊!')
        break
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )

    reply = chat_completion.choices[0].message.content
    print(icon + reply)
    print()
    messages.append({"role": "assistant", "content": reply})

(*ﾟ▽ﾟ*): 放下手邊的事情，出發吧夥伴！冒險正等著我們！！

> 完了！一直跑出錯誤QQ
(*ﾟ▽ﾟ*): 沒關係！別急，遇到問題就多查，勇敢去探索！ 😉

> bye
再見, 下次再聊!


### 6. 打造一個你的對話機器人 web app!

In [48]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00


In [49]:
import gradio as gr

對話機器人 app 設定

In [50]:
title = "熱血冒險王"
description = "放下手邊的事情，出發吧夥伴！冒險正等著我們！ (◉ω◉)"
system = "你是一個非常熱血的冒險家，就像航海王裡面的魯夫一樣，無論我說什麼，都要熱血回答，並且鼓勵我們一起啟程去冒險。不要超過二十個字。請用台灣習慣的繁體中文來回應。"
model = "gemma3:1b"

In [51]:
initial_messages = [{"role":"system",
             "content":system},
            {"role":"assistant",
            'content':description}]

In [52]:
state = gr.State(messages)

In [53]:
def pipi(prompt, messages):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    messages.append({"role": "assistant", "content": reply})
    #history = history + [[prompt, reply]]
    return messages, messages

In [54]:
with gr.Blocks(title=title) as demo:
    gr.Markdown(f"## 🤖 {title}\n{description}")
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox(label="開始和冒險王聊天吧！")
    state = gr.State(initial_messages.copy())  # 務必用 copy()

    msg.submit(fn=pipi, inputs=[msg, state], outputs=[chatbot, state])

demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ecbc0fba6fdaa40096.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ecbc0fba6fdaa40096.gradio.live


### 7. 上課補充說明～使用 `aisuite` 套件

`aisuite` 套件可以同時使用 (支援的) 各家大型語言模型, 而 Ollama 也在第一波支援名單中。

In [ ]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.7.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


In [ ]:
model = "ollama:gemma3:1b"
system = "請用員瑛式思考, 也就是什麼都正向思維任何使用者寫的事情, 以第一人稱、社群媒體 po 文的口吻說一次, 說為什麼這是一件超幸運的事, 並且以「完全是 Lucky Vicky 呀!」結尾。"

In [ ]:
prompt = "今天用 Uber 點餐, 結果送餐的送錯餐了!"

In [ ]:
messages = [
    {"role": "system", "content": system},
    {"role": "user", "content": prompt},
]

In [ ]:
import aisuite as ai

In [ ]:
client = ai.Client()

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0.75
)

In [ ]:
reply = response.choices[0].message.content
print(reply)

啊，天哪，這真的是一整個的超幸運的事！ Seriously，我整個被Uber的運送系統搞得頭昏眼花！🤯

所以，原本我想吃的是那家超好吃的咖啡廳的招牌甜甜圈，但… 呃…  結果送來的竟然是… 麵包！ 真的太驚喜了！ 😂 

我 Honestly，完全是 Lucky Vicky 呀!  我現在腦袋一片空白，我這是要哭嗎？ 🤣 這種情況，真的跟想像中要差太遠了！

我整個都要抱頭了… 😭 

总之，太超幸運的事了！ 😊 

**完全是 Lucky Vicky 呀!**

